In [16]:
import pandas as pd
import heapq
from math import radians, cos, sin, sqrt, atan2

file_path = 'C:/Users/Vzhan/OneDrive/Bureau/Master/Epitech/TravelOrder/Travel-Order-Resolver/ai/path_algorithm/dataset/liste-des-gares.csv'

# Load the dataset and construct the graph
def load_graph_from_ratp_csv(file_path):
    ratp_data = pd.read_csv(file_path, delimiter=';')
    graph = {}

    # Iterate through the dataset
    for _, row in ratp_data.iterrows():
        source = row['LIBELLE']
        source_coords = (row['Y_WGS84'], row['X_WGS84'])  # Latitude, Longitude
        
        # Find all potential connections based on CODE_LIGNE or other criteria
        connections = ratp_data[ratp_data['CODE_LIGNE'] == row['CODE_LIGNE']]
        
        #print(f"Source: {source} (Coordinates: {source_coords})")
        
        for _, conn_row in connections.iterrows():
            destination = conn_row['LIBELLE']
            destination_coords = (conn_row['Y_WGS84'], conn_row['X_WGS84'])
            
            # Avoid self-loops
            if source != destination:
                weight = haversine(source_coords, destination_coords)
                #print(f"Adding connection from {source} to {destination} with weight {weight:.2f} km")
                graph.setdefault(source, []).append((destination, weight))
                graph.setdefault(destination, []).append((source, weight))
    
    return graph
    
# Inspect the first few rows
graph = load_graph_from_ratp_csv(file_path)
#print("Graph:", graph)

# Dijkstra's algorithm to find the shortest path
def dijkstra_path(graph, start, end):
    if start not in graph:
        #print(f"Start station '{start}' not found in graph.")
        return float('inf'), []
    if end not in graph:
        #print(f"End station '{end}' not found in graph.")
        return float('inf'), []
    
    priority_queue = [(0, start, [])]  # (distance, node, path)
    visited = set()
    
    while priority_queue:
        current_distance, current_node, path = heapq.heappop(priority_queue)
        
        if current_node in visited:
            continue
        visited.add(current_node)
        
        # Update the path
        path = path + [current_node]
        
        # If the destination is reached, return the path and distance
        if current_node == end:
            return current_distance, path
        
        for neighbor, weight in graph.get(current_node, []):
            if neighbor not in visited:
                heapq.heappush(priority_queue, (current_distance + weight, neighbor, path))
    
    return float('inf'), []  # If no path exists

def haversine(coord1, coord2):
    R = 6371  # Earth radius in kilometers
    lat1, lon1 = radians(coord1[0]), radians(coord1[1])
    lat2, lon2 = radians(coord2[0]), radians(coord2[1])
    
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    
    return R * c

# Test Dijkstra's algorithm for LIBELLE to LIBELLE
start_station = "Ermont-Eaubonne"  # Replace with the LIBELLE of the starting station
end_station = "La Défense"      # Replace with the LIBELLE of the destination station
distance, path = dijkstra_path(graph, start_station, end_station)

print(f"Shortest path from {start_station} to {end_station}: {path} with distance {distance:.2f} km")

Shortest path from Ermont-Eaubonne to La Défense: ['Ermont-Eaubonne', 'Péreire-Levallois', 'Pont-Cardinet', 'La Défense'] with distance 17.45 km
